In [ ]:
cd ../

In [2]:
import pickle
import numpy as np
import argparse

import mat73
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.layers import Dense, Input, Flatten, Add, Multiply, Lambda
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

from utils.explanations import calculate_robust_astute_sampled
import shap

np.random.seed(0)

/home/zulqarnain/anaconda3/envs/old_tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zulqarnain/anaconda3/envs/old_tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zulqarnain/anaconda3/envs/old_tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zulqarnain/anaconda3/envs/ol

In [3]:
def shap_explainer(datatype, ball_r, epsilon, prop_points, exponentiate, classifier):
    blackbox_path = 'models/' + datatype + '_blackbox.hdf5'
    cifar = pickle.load(open('data/cifar10_6_6_train.pkl', 'rb'))
    data = cifar[1]
    labels = cifar[0]
    x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)
    x_train, x_val = np.array(x_train), np.array(x_val)
    x_train, x_val = x_train.reshape((len(x_train), -1)), x_val.reshape((len(x_val), -1))
    y_train_orig, y_val_orig = y_train.copy(), y_val.copy()
    y_train, y_val = to_categorical(y_train), to_categorical(y_val)
    input_shape = x_train.shape[-1]
    
    if classifier == '2layer':
        activation = 'relu'

        model_input = Input(shape=(input_shape,), dtype='float32')

        net = Dense(32, activation=activation, name='dense1',
                    kernel_regularizer=regularizers.l2(1e-3))(model_input)

        preds = Dense(10, activation='softmax', name='dense3',
                      kernel_regularizer=regularizers.l2(1e-3))(net)
        bbox_model = Model(model_input, preds)

        bbox_model.load_weights('models/' + datatype + '_blackbox.hdf5',
                                by_name=True)
        pred_model = Model(model_input, preds)

    elif classifier == '4layer':
        activation = 'relu'

        model_input = Input(shape=(input_shape,), dtype='float32')

        net = Dense(32, activation=activation, name='dense1',
                    kernel_regularizer=regularizers.l2(1e-3))(model_input)
        net = Dense(32, activation=activation, name='dense2',
                    kernel_regularizer=regularizers.l2(1e-3))(net)
        net = Dense(32, activation=activation, name='dense3',
                    kernel_regularizer=regularizers.l2(1e-3))(net)
        net = Dense(32, activation=activation, name='dense4',
                    kernel_regularizer=regularizers.l2(1e-3))(net)
        preds = Dense(10, activation='softmax', name='dense5',
                      kernel_regularizer=regularizers.l2(1e-3))(net)
        bbox_model = Model(model_input, preds)
        bbox_model.load_weights('models/' + datatype + '_blackbox_extra.hdf5',
                                by_name=True)
        pred_model = Model(model_input, preds)


    elif classifier == 'linear':
        activation = None

        model_input = Input(shape=(input_shape,), dtype='float32')

        net = Dense(32, activation=activation, name='dense1',
                    kernel_regularizer=regularizers.l2(1e-3))(model_input)

        preds = Dense(10, activation='softmax', name='dense3',
                      kernel_regularizer=regularizers.l2(1e-3))(net)
        bbox_model = Model(model_input, preds)
        bbox_model.load_weights('models/' + datatype + '_blackbox_linear.hdf5',
                                by_name=True)
        pred_model = Model(model_input, preds)


    elif classifier == 'svm':
        pred_model = pickle.load(open('models/' + datatype + '_svm.pk', 'rb'))

    if classifier == 'svm':
#         training_indices = np.random.choice(len(x_train), int(0.001*len(x_train)), replace=False)
        explainer = shap.KernelExplainer(pred_model.predict_proba, shap.kmeans(x_train, 100))


        explanation = calculate_robust_astute_sampled(data=x_val,
                                                   explainer=explainer,
                                                   explainer_type='shap',
                                                   explanation_type='attribution',
                                                   ball_r=ball_r,
                                                   epsilon=epsilon,
                                                   num_points=int(prop_points * len(x_val)),
                                                   exponentiate=exponentiate,
                                                   calculate_astuteness=False,
                                                   NN=False)
    else:
        background = x_train[np.random.choice(len(x_train), 100, replace=False)]
        explainer = shap.GradientExplainer(bbox_model, background)

        explanation = calculate_robust_astute_sampled(data=x_val,
                                                      explainer=explainer,
                                                      explainer_type='shap',
                                                      explanation_type='attribution',
                                                      ball_r=ball_r,
                                                      epsilon=epsilon,
                                                      num_points=int(prop_points * len(x_val)),
                                                      exponentiate=exponentiate,
                                                      calculate_astuteness=False)

    del pred_model
    return np.abs(explanation)


In [4]:
ball_radius = 2
epsilon = 0.05
prop_points = 0.05
run_times = 5
exponentiate = 0
classifiers = ['2layer', 'linear', '4layer', 'svm']

for datatype in ['cifar1006']:
    for c in range(len(classifiers)):
        for i in range(run_times):
            fname = 'explained_weights/shap/' + 'shap_' + datatype + '_' + classifiers[c] + '_' + str(i) + '.gz'
            explanation = shap_explainer(datatype=datatype,
                                           ball_r=ball_radius,
                                           epsilon=epsilon,
                                           prop_points=prop_points,
                                           exponentiate=exponentiate,
                                           classifier=classifiers[c])
            np.savetxt(X=explanation, fname=fname, delimiter=',')


Instructions for updating:
Colocations handled automatically by placer.


2022-05-18 11:47:34.513317: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-18 11:47:34.516385: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3600000000 Hz
2022-05-18 11:47:34.516850: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55978e09b300 executing computations on platform Host. Devices:
2022-05-18 11:47:34.516866: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Using TensorFlow backend.


  0%|          | 0/951 [00:00<?, ?it/s]

  0%|          | 0/951 [00:00<?, ?it/s]

  0%|          | 0/951 [00:00<?, ?it/s]

  0%|          | 0/951 [00:00<?, ?it/s]

  0%|          | 0/951 [00:00<?, ?it/s]